In [23]:
#!/usr/bin/env python
# coding: utf-8
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from copy import deepcopy
from matplotlib import pyplot as plt
import neurokit as nk
import random
import datetime
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=UserWarning)
from interpret import ecgInterpretation
from data_for_test import get_data_for_test
import pandas as pd
import numpy as np
import sys
import h5py


In [28]:
with h5py.File('data/ecg_tracings.hdf5', 'r') as file:
        data = np.array(file['tracings'])
y = pd.read_csv('data/annotations/gold_standard.csv')

In [49]:
y[(y['1dAVb'] == 1) | (y['RBBB'] == 1) | (y['LBBB'] == 1) | (y['SB'] == 1) | (y['AF'] == 1) | (y['ST'] == 1)]

,1dAVb,RBBB,LBBB,SB,AF,ST
1,0,0,1,0,0,0
12,1,0,0,0,0,0
15,1,0,1,0,0,0
18,1,0,1,0,0,0
23,0,0,0,0,0,1
28,0,0,1,0,0,0
32,1,0,0,0,0,0
33,0,0,0,0,0,1
40,0,0,0,1,0,0
57,1,0,0,0,0,0


In [2]:
val_path = '/scratch/derickmath/datasets/base_dados_laudos_unificada_revista.csv'
val_traces = '/scratch/derickmath/datasets/val_traces.csv'
# Get data
data_ori = pd.read_csv(val_traces, sep = ";")
exames = pd.read_csv(val_path, sep = ",")
pd.options.display.max_columns = None
data = get_data_for_test(path_to_traces=val_traces,
                               path_to_val=val_path)
signals =  np.array([x for x in data['x'][:][0]])
datax = data['x']
datay = data['y']

In [29]:
[i for i,x in enumerate(y) if x > 0]

AttributeError: 'str' object has no attribute 'any'

In [249]:
%matplotlib inline
import mpld3

ecg_id = 1

data = datax[ecg_id]
signal = [None] * 12
ecg_process = [None] * 12
for i in range(12):
    signal[i] = np.array([x[i] for x in data])
    ecg_process[i] = nk.ecg_process(signal[i],sampling_rate = 400,hrv_features = None,filter_type='FIR')
    ecg_process[i]['ECG']['R_Peaks'] = [[y, x] for x, y in enumerate(ecg_process[i]['ECG']['R_Peaks'])]
peaks = ecg_process
ECG = np.transpose(data)

mpld3.enable_notebook()
derivations = ["DI", "DII", "DIII", "AVL", "AVF", "AVR", "V1", "V2", "V3", "V4", "V5", "V6"]

fig, ax = plt.subplots(nrows=1, ncols=3, figsize=(20, 5), gridspec_kw={'width_ratios': [4, 1, 1]})
fig.tight_layout(rect=[-0.015, -0.015, 0.9, 0.95])

beg = 999
ders = [1, 6, 11]
incs = [2000, 1000, 1000]
time = [5, 2.5, 2.5]
xpos = [0.95,0.8,0.8]

for i in range(3):

    end = beg + incs[i]
    der = ders[i]
#     der = 0
    p = np.array([x[0] - beg for x in peaks[der]['ECG']['P_Waves'] if x[0] <= end and x[0] >= beg])
    q = np.array([x[0]- beg for x in peaks[der]['ECG']['Q_Waves'] if x[0] <= end and x[0] >= beg])
    r = np.array([x[0]- beg for x in peaks[der]['ECG']['R_Peaks'] if x[0] <= end and x[0] >= beg])
    s = np.array([x[0]- beg for x in peaks[der]['ECG']['S_Waves'] if x[0] <= end and x[0] >= beg])
    t = np.array([x[0]- beg for x in peaks[der]['ECG']['T_Waves_Onsets'] if x[0] <= end and x[0] >= beg])
    t_ = np.array([x[0]- beg for x in peaks[der]['ECG']['T_Waves_Ends'] if x[0] <= end and x[0] >= beg])

    x = np.arange(incs[i])

    ax[i].plot(x/400, ECG[der][beg:end]/5, '-k', linewidth=2.5)
    ax[i].plot(p/400, ECG[der][p + beg]/5, 'rv', ms = 8)
    ax[i].plot(q/400, ECG[der][q + beg]/5, 'g^', ms = 8)
    ax[i].plot(r/400, ECG[der][r + beg]/5, 'y<', ms = 8)
    ax[i].plot(s/400, ECG[der][s + beg]/5, 'm>', ms = 8)
    ax[i].plot(t/400, ECG[der][t + beg]/5, 'cs', ms = 8)
    ax[i].plot(t_/400, ECG[der][t_+ beg]/5, 'co', ms = 8)
    major_ticksy = np.arange(-1.45, 1.45, 0.5)
    minor_ticksy = np.arange(-1.45, 1.45, 0.1)
    major_ticksx = np.arange(0, time[i], 0.2)
    minor_ticksx = np.arange(0, time[i], 0.05)
    ax[i].set_xlim(0, time[i])
    ax[i].grid(which='both')
    ax[i].set_xticks(major_ticksx)
    ax[i].set_xticks(minor_ticksx, minor=True)
    ax[i].set_yticks(major_ticksy)
    ax[i].set_yticks(minor_ticksy, minor=True)
    ax[i].grid(which='minor', alpha=0.2, color='r')
    ax[i].grid(which='major', alpha=0.5, color='r')
    ax[i].set_xticklabels( () )
    ax[i].set_yticklabels( () )
#     der = ders[i]
    ax[i].set_title(derivations[der], fontsize=28, x = xpos[i])
    plt.legend(["signal", "p", "q", "r", "s", "t begin", "t end"], loc=4, fancybox=True, shadow=True, prop={'size': 22}, bbox_to_anchor=(1.869, 0.09))

plt.savefig("plts_chil/" + str(ecg_id) + ".pdf")
plt.close()